In [6]:
import os
import re
import pandas as pd
import requests
import csv
from urllib.parse import quote

# Configura tu API key de OMDb
API_KEY = "4348731d"  # Reemplaza con tu API key

# Ruta al archivo movies.dat (ajusta la ruta según tu estructura)
movies_file = "../data/ml-1m/ml-1m/movies.dat"

# Función para limpiar el título y extraer el año (si existe)
def clean_title_and_extract_year(title):
    # Busca un patrón tipo "Título (Año)"
    m = re.search(r"^(.*)\((\d{4})\)$", title.strip())
    if m:
        title_clean = m.group(1).strip()
        year = m.group(2).strip()
    else:
        title_clean = title.strip()
        year = ""
    return title_clean, year

# Cargar movies.dat
movies_df = pd.read_csv(movies_file, sep="::", engine="python", header=None, 
                          names=["MovieID", "Title", "Genres"], encoding="latin-1")
print(f"Total de películas en movies.dat: {len(movies_df)}")

# Archivo CSV donde se guardarán los datos de OMDb
csv_file = "omdb_movie_data.csv"

# Cargar títulos ya scrapeados, si existe el CSV
if os.path.exists(csv_file):
    scraped_df = pd.read_csv(csv_file, encoding="utf-8")
    scraped_titles = set(scraped_df["Title"].str.lower())
else:
    scraped_titles = set()

# Abrir el CSV en modo "append" (si no existe, se crea y se escribe el header)
csv_exists = os.path.exists(csv_file)
with open(csv_file, "a", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["MovieID", "Title", "IMDbRating", "Plot", "Director", "Actors", "Genre", "Runtime", "Released"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if not csv_exists:
        writer.writeheader()
    
    count = 0
    # Iterar sobre todas las películas de movies.dat
    for idx, row in movies_df.iterrows():
        movie_id = row["MovieID"]
        original_title = row["Title"]
        # Limpiar el título y extraer el año
        title_clean, year = clean_title_and_extract_year(original_title)
        
        # Verificar si ya se scrapeó (usando el título limpio en minúsculas)
        if title_clean.lower() in scraped_titles:
            continue
        
        # Construir la URL de la API, incluyendo el año si está disponible
        url = f"http://www.omdbapi.com/?apikey={API_KEY}&t={quote(title_clean)}"
        if year:
            url += f"&y={year}"
            
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data.get("Response") == "True":
                result = {
                    "MovieID": movie_id,
                    "Title": data.get("Title", "N/A"),
                    "IMDbRating": data.get("imdbRating", "N/A"),
                    "Plot": data.get("Plot", "N/A"),
                    "Director": data.get("Director", "N/A"),
                    "Actors": data.get("Actors", "N/A"),
                    "Genre": data.get("Genre", "N/A"),
                    "Runtime": data.get("Runtime", "N/A"),
                    "Released": data.get("Released", "N/A")
                }
                writer.writerow(result)
                print(f"Scraped: {result['Title']} - IMDbRating: {result['IMDbRating']}")
                scraped_titles.add(title_clean.lower())
                count += 1
                # Limitar a 1000 películas nuevas
                if count >= 1000:
                    break
            else:
                print(f"OMDb API no encontró datos para: {title_clean} ({data.get('Error')})")
        else:
            print(f"Error en la solicitud para: {title_clean}")
            
print("Scraping completado. Datos guardados en 'omdb_movie_data.csv'.")


Total de películas en movies.dat: 3883
OMDb API no encontró datos para: American President, The (Movie not found!)
OMDb API no encontró datos para: City of Lost Children, The (Movie not found!)
OMDb API no encontró datos para: Shanghai Triad (Yao a yao yao dao waipo qiao) (Movie not found!)
OMDb API no encontró datos para: Twelve Monkeys (Movie not found!)
OMDb API no encontró datos para: Seven (Se7en) (Movie not found!)
OMDb API no encontró datos para: Usual Suspects, The (Movie not found!)
OMDb API no encontró datos para: Big Green, The (Movie not found!)
OMDb API no encontró datos para: Postino, Il (The Postman) (Movie not found!)
OMDb API no encontró datos para: Confessional, The (Le Confessionnal) (Movie not found!)
OMDb API no encontró datos para: Indian in the Cupboard, The (Movie not found!)
Scraped: Lawnmower Man 2: Beyond Cyberspace (1996) - Episode 21 BARGAIN BIN - IMDbRating: N/A
OMDb API no encontró datos para: French Twist (Gazon maudit) (Movie not found!)
OMDb API no enc

KeyboardInterrupt: 